In [159]:
# !python -m spacy download en_core_web_md

from config.constants import RELATIVE_DATA_PATH, FILE_NAME
from pathlib import Path
from config.constants import RELATIVE_DATA_PATH, FILE_NAME
from pathlib import Path
import pandas as pd
import re

sbar = "========================"
lbar = "===================================================="
lbreak = "\n\n\n\n"

ROOT = Path().cwd() 
file_path = ROOT / RELATIVE_DATA_PATH / FILE_NAME


def get_raw_corpus(file_path):
    
    with open(file_path, 'r', encoding='utf-8') as f:
        docs = f.readlines()
    
    # remove first element
    docs.pop(0)
        
    return docs

    
docs = get_raw_corpus(file_path=file_path)
df = pd.DataFrame(docs, columns=['raw_docs'])


df = df.assign(clean_docs = lambda d: d['raw_docs']
                                                   .str.replace('\n', '', regex=True)  # remove \n
                                                   .str.strip())                       # strip each document
df.head()

,raw_docs,clean_docs
0,Michael Novogratz is the founder of finance co...,Michael Novogratz is the founder of finance co...
1,Goldman Sachs has over-the-counter (OTC) crypt...,Goldman Sachs has over-the-counter (OTC) crypt...
2,Goldman Sachs and trading unit of Michael Novo...,Goldman Sachs and trading unit of Michael Novo...
3,over-the-counter (OTC) crypto options was in t...,over-the-counter (OTC) crypto options was in t...
4,Max Minton is Asia Pacific head of digital ass...,Max Minton is Asia Pacific head of digital ass...


In [160]:


# strip docuiments
docs = df['clean_docs'].to_list()


for doc in docs[:3]:
    print(lbar)
    print(doc)
    print(lbar)
    print("\n\n\n")

Michael Novogratz is the founder of finance company Galaxy Digital.




Goldman Sachs has over-the-counter (OTC) crypto options.




Goldman Sachs and trading unit of Michael Novogratz’s cryptocurrency investment management firm Galaxy Digital have over-the-counter (OTC) crypto options.






In [161]:
import spacy
from spacy import displacy
nlp = spacy.load("en_core_web_md")

In [162]:
for doc in nlp.pipe(docs[:4]):
    print(lbar)
    displacy.render(doc, style='ent')
    print(lbar)
    print(lbreak)

/home/sergio/Documents/repos/NERtropy/.venv/lib/python3.8/site-packages/spacy/displacy/__init__.py:205: UserWarning: [W006] No entities to visualize found in Doc object. If this is surprising to you, make sure the Doc was processed using a model that supports named entity recognition, and check the `doc.ents` property manually if necessary.
  warnings.warn(Warnings.W006)


Calculemos las frecuencias de las entidades que tenemos

In [185]:
doc = nlp(text)
ents = doc.ents
ents[0].text

'Michael Novogratz'

In [189]:
from collections import Counter

def extract_ent_type(text):
    doc = nlp(text)
    ents = doc.ents
    if len(ents) > 0:
        ents_type = [ent.label_ for ent in ents]
    else:
        ents_type = []
    return ents_type

def extract_ent(text):
    doc = nlp(text)
    ents = doc.ents
    if len(ents) > 0:
        ents = [ent.text.lower() for ent in ents]
    else:
        ents = []
    return ents

df = (df
        .assign(ent_types = lambda d: d['clean_docs'].apply(lambda x: extract_ent_type(x)))
        .assign(ents = lambda d: d['clean_docs'].apply(lambda x: extract_ent(x)))
     )
df.head()

,raw_docs,clean_docs,ent_types,ents
0,Michael Novogratz is the founder of finance co...,Michael Novogratz is the founder of finance co...,"[PERSON, ORG]","[michael novogratz, galaxy digital]"
1,Goldman Sachs has over-the-counter (OTC) crypt...,Goldman Sachs has over-the-counter (OTC) crypt...,[ORG],[goldman sachs]
2,Goldman Sachs and trading unit of Michael Novo...,Goldman Sachs and trading unit of Michael Novo...,"[ORG, PERSON, ORG]","[goldman sachs, michael novogratz’s, galaxy di..."
3,over-the-counter (OTC) crypto options was in t...,over-the-counter (OTC) crypto options was in t...,[],[]
4,Max Minton is Asia Pacific head of digital ass...,Max Minton is Asia Pacific head of digital ass...,"[PERSON, LOC, ORG]","[max minton, asia pacific, goldman sachs]"


In [219]:
what_count = 'ent_types'
c = Counter(
    [
        ent
        for l in df[what_count]
        for ent in l
    ]
)
c.most_common(5)

[('ORG', 49), ('PERSON', 14), ('CARDINAL', 4), ('GPE', 4), ('PRODUCT', 3)]

parece razonable extraer ORG y en todo caso PERSON